In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
cd cdQA-bert_qa_vGPU

/content/drive/MyDrive/cdQA-bert_qa_vGPU


In [5]:
pip install -e .

Obtaining file:///content/drive/MyDrive/cdQA-bert_qa_vGPU
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 12.2 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=4218c5b6b08f5d6b25e0685aba2871433d941c6ead8d0896d4ed130369b77e27
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling 

In [6]:
pip install cdqa

In [7]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline.cdqa_sklearn import QAPipeline

/content/drive/MyDrive/cdQA-bert_qa_vGPU/cdqa/reader/bertqa_sklearn.py:36: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
df = pd.read_csv('./data/english.csv', encoding='cp949',converters={'paragraphs': literal_eval})

df = filter_paragraphs(df)
df['content'] = df['paragraphs'].apply(lambda x: ' '.join(x))

In [9]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa_vGPU-sklearn.joblib') # bert_qa_squad_vGPU.joblib / bert_qa_vGPU-sklearn.joblib
cdqa_pipeline.fit(X=df)
cdqa_pipeline.reader.output_dir = './logs/'

In [10]:
query = 'Where is the College of Hummanites?'
prediction = cdqa_pipeline.predict(X=query)

+------+-------+-----------------------------------+
| rank | index |               title               |
+------+-------+-----------------------------------+
|  1   |   1   | College of Information Technology |
|  2   |   0   |       College of Humanities       |
|  3   |   2   |          Student ID Card          |
+------+-------+-----------------------------------+
Time: 0.00372 seconds


3it [00:00, 2067.18it/s]
100%|██████████| 231508/231508 [00:00<00:00, 358476.09B/s]


In [11]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))

query: Where is the College of Hummanites?
answer: INU


In [12]:
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

title: Student ID Card
paragraph: Please remember that procedures and policies may change over time, so it's always a good idea to consult directly with the Shinhan Bank Business Office at INU for the most accurate and up-to-date information regarding the process of obtaining and using your student ID card as a bank check card on campus
